In [1]:
%run 01_Credentials.ipynb
import os 
import random
import datetime
import psycopg2
from datetime import datetime

In [2]:
data_sheet = \
{'T': {'Sensor_Type': 'Temperature', 'Manufacturer': 'SHT31', 'Model': 'SHT31-DIS', 'Unit': '°C'}, 
 'H': {'Sensor_Type': 'Humidity', 'Manufacturer': 'SHT31', 'Model': 'SHT31-DIS', 'Unit': '%RH'}, 
 'CO2': {'Sensor_Type': 'CO2', 'Manufacturer': 'CO2-BME688', 'Model': 'BME688', 'Unit': 'ppm'}, 
 'SM': {'Sensor_Type': 'Soil_Moisture', 'Manufacturer': 'EC-5', 'Model': 'EC-5', 'Unit': '%VWC'}, 
 'PH': {'Sensor_Type': 'Soil_pH', 'Manufacturer': 'PH-20', 'Model': 'PH-20', 'Unit': 'pH'}, 
 'L': {'Sensor_Type': 'Light', 'Manufacturer': 'BH1750', 'Model': 'BH1750FVI', 'Unit': 'lux'}}

In [ ]:
# data_simulator is a function that generate sensor data 
# takes number of Greenhouses locations as an input to generate these data
def data_simulator(num_of_GH_location):
    n = int(num_of_GH_location)
    
    sensors_metadat = {
        "T":["Temperature", float(round(random.uniform(20,42),2))],
        "H":["Humidity", float(round(random.uniform(20,100),1))],
        "CO2":["CO2",random.randint(400,1800)],
        "SM":["Soil_Moisture", float(round(random.uniform(50,100),1))],
        "PH":["Soil_pH",float(round(random.uniform(3,10),1))],
        "L":["Light",random.randint(1200,1400)] }
    
    reandom_key = random.choice(list(sensors_metadat.keys()))
    suffix = random.randint(1,n)

    
    sensor_data = {
        "sensor_id": "{}_{}".format(reandom_key,suffix) ,
        "timestamp": str(datetime.now()),
        "sensor_type": sensors_metadat[reandom_key][0],
        "value": sensors_metadat[reandom_key][1],
        "location": "GH_{}".format(suffix) } 
    
    return sensor_data

In [4]:
# get_time_parts is a function get_time_parts Splits a timestamp string 
# into its fundamental parts: weekday,month,day,hour,minute,second,year    
def get_time_parts(time):
    try:
        time_obj = datetime.strptime(time,  "%Y-%m-%d %H:%M:%S.%f")
        weekday ={
            0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday",
            4:"Friday", 5:"Saturday", 6:"Sunday"}
        
        return (time,weekday[time_obj.weekday()],time_obj.month,
                time_obj.day,time_obj.hour,time_obj.minute,
                time_obj.second,time_obj.year)
    except ValueError:
        print("Invalid time string format. Expected format: 'Weekday Month Day HH:MM:SS Year'")
        return None

In [5]:
def connect_to_db (db_name):
    try:
        print("Connecting to Billing_DWH..... ")
        conn = psycopg2.connect(
            user = os.environ.get('user_name'),
            password = os.environ.get('Password_GH_Sensors_DWH'),
            host = os.environ.get('db_host_name'),
            port = os.environ.get('db_port'),
            database = db_name
        )
        print ("Connection to PostgerSQL is done successfuly") 
        cur = conn.cursor()
        print ("conn and cur are created and successfuly returned")
        return conn ,cur
    except:
        print("Cann't connect to PostgerSQL")
 

In [6]:
def insert_into_time_dimension(timestamp,cur,conn):
    data_to_time_dimension = "insert into time_dimension values {} ON CONFLICT (timestamp) DO NOTHING;".format(get_time_parts(timestamp))
    cur.execute(data_to_time_dimension)
    conn.commit()

In [7]:
def insert_into_location_dimension(location,conn,cur):
    l_n = int(location.split("_")[-1])
    description = "Greenhouse number {} on the farm".format(l_n)
    location_dimension = "insert into location_dimension values {} ON CONFLICT (location_id) DO NOTHING;".format((location,description))
    cur.execute(location_dimension)
    conn.commit()

In [8]:
def insert_into_sensor_dimension(sensor_id,conn,cur):
    Sensor_sympol = str(sensor_id).strip().split("_")[0]
    data = [sensor_id] + list(data_sheet[Sensor_sympol].values())
    sensor_dimension = "insert into sensor_dimension values {} ON CONFLICT (sensor_id) DO NOTHING;".format(tuple(data))
    cur.execute(sensor_dimension)
    conn.commit()

In [9]:
def insert_into_sensor_measurements(message,conn,cur):
    data = tuple (message.values())
    sensor_measurements = "insert into sensor_measurements(sensor_id,timestamp,sensor_type,value,location_id) values {}".format(data)
    cur.execute(sensor_measurements)
    conn.commit()